# LeTour data set 
This file downloads raw data about every rider of every Tour de France (from 1903 up to 2020). This data will then be postprocessed and stored in CSV format.
Executing this notebook might take some minutes.

In [ ]:
!pip install -r requirements.txt

## 1) Retrieve urls for data extract
First we generate the urls that we need to download the raw HTML pages from the `letour.fr` website to work offline from here on. The dataframe dflink will stores the respective url for each year.
Take a look at `view-source:https://www.letour.fr/en/history` (at around line 1143-1890).  

In [59]:
import collections
import datetime
import os
import re
import subprocess
import time
from io import StringIO
from pathlib import Path

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [60]:
PREFIX = "http://www.letour.fr"
HISTORYPAGE = "https://www.letour.fr/en/history"

In [61]:
headers = {
    "Accept": "text/html",
    "User-Agent": "python-requests/1.2.0",
    "Accept-Charset": "utf-8",
    "accept-encoding": "deflate, br",
}

In [62]:
resulthistpage = requests.get(HISTORYPAGE, allow_redirects=True, headers=headers)
souphistory = BeautifulSoup(resulthistpage.text, "html.parser")

In [63]:
# Find all option elements with the data-tabs-ajax attribute
options = souphistory.find_all('option', {'data-tabs-ajax': True})

# Extract the data-tabs-ajax attributes
data_tabs_ajax_values = [option['data-tabs-ajax'] for option in options]

# Create a DataFrame with the extracted values
dflink = pd.DataFrame(data_tabs_ajax_values, columns=['TDFHistorylink'])
dflink

,TDFHistorylink
0,/en/block/history/11823/4e4949d130f21278f7d8bc...
1,/en/block/history/11822/ca646e1b4b99d7ee973c45...
2,/en/block/history/11821/a3fe7d350ef546e3c27ae8...
3,/en/block/history/11820/17fa8e795e69e9f326ea26...
4,/en/block/history/11819/96c0eb3fa403ebf222f28b...
...,...
105,/en/block/history/10711/f558d3bc819c8ee5dc627d...
106,/en/block/history/10710/ead1d1704b1600c795619e...
107,/en/block/history/10709/c5f53ced72a23f333cc186...
108,/en/block/history/10708/0b76b8f809ad5d8bcf3579...


## 2) Get data from HTML pages and convert results to dataframes

### 2.1) Create function that convert HHh mm' ss'' to seconds

In [64]:
def calcTotalSeconds(row, mode):
    val = sum(
        x * int(t)
        for x, t in zip(
            [3600, 60, 1],
            row.replace("h", ":")
            .replace("'", ":")
            .replace('"', ":")
            .replace(" ", "")
            .replace("+", "")
            .replace("-", "0")
            .split(":"),
        )
    )

    if (mode == "Gap") and val > 180000:
        val = 0

    return val

### 2.2) Create a function that will retrieve elements from a source HTML page located on an input url

In [65]:
def getstagesNrank(i_url):
    resultfull = requests.get(i_url, allow_redirects=True)
    result = resultfull.text
    resultstatus = resultfull.status_code

    print(i_url + " ==> HTTP STATUS = " + str(resultstatus))

    soup = BeautifulSoup(result, "html.parser")
    h = soup.find("h3")
    year = int(h.text[-4:])

    # Find select tag
    select_tag = soup.find("select")

    # find all option tag inside select tag
    options = select_tag.find_all("option")

    cols = ["Year", "TotalTDFDistance", "Stage"]
    lst = []

    # search for stages
    distance = soup.select("[class~=statsInfos__number]")[1].contents

    # search for distance of the TDF edition
    for option in options:
        lst.append([year, int(distance[0].replace(" ", "")), option.text])

    dfstages = pd.DataFrame(lst, columns=cols)

    # Find select tag for ranking racers
    rankingTable = soup.find("table")

    # Wrap the HTML string in a StringIO object
    html_io = StringIO(str(rankingTable))

    # Use pd.read_html with the StringIO object
    dfrank = pd.read_html(html_io)[0]

    dfrank["Year"] = year
    dfrank["Distance (km)"] = int(distance[0].replace(" ", ""))
    dfrank["Number of stages"] = len(dfstages)
    dfrank["TotalSeconds"] = dfrank["Times"].apply(
        lambda x: calcTotalSeconds(x, "Total")
    )
    dfrank["GapSeconds"] = dfrank["Gap"].apply(lambda x: calcTotalSeconds(x, "Gap"))

    return dfstages, dfrank

In [ ]:
def getstagesNrank(i_url):
    resultfull = requests.get(i_url, allow_redirects=True)
    result = resultfull.text
    resultstatus = resultfull.status_code

    print(i_url + " ==> HTTP STATUS = " + str(resultstatus))

    soup = BeautifulSoup(result, "html.parser")
    h = soup.find("h3")
    year = int(h.text[-4:])

    # Find select tag
    select_tag = soup.find("select")

    # find all option tag inside select tag
    options = select_tag.find_all("option")

    cols = ["Year", "TotalTDFDistance", "Stage"]
    lst = []

    # search for stages
    distance = soup.select("[class~=statsInfos__number]")[1].contents

    # search for distance of the TDF edition
    for option in options:
        lst.append([year, int(distance[0].replace(" ", "")), option.text])

    dfstages = pd.DataFrame(lst, columns=cols)

    # Find select tag for ranking racers
    rankingTable = soup.find("table")

    dfrank = pd.read_html(str(rankingTable))[0]

    dfrank["Year"] = year
    dfrank["Distance (km)"] = int(distance[0].replace(" ", ""))
    dfrank["Number of stages"] = len(dfstages)
    dfrank["TotalSeconds"] = dfrank["Times"].apply(
        lambda x: calcTotalSeconds(x, "Total")
    )
    dfrank["GapSeconds"] = dfrank["Gap"].apply(lambda x: calcTotalSeconds(x, "Gap"))

    return dfstages, dfrank

### 2.3) Loop on the dflink dataframe to get data from each url source

In [69]:
# Reverse the DataFrame to process the links in the desired order
dflink_reversed = dflink.iloc[::-1].reset_index(drop=True)

dflink_reversed

,TDFHistorylink
0,/en/block/history/10707/d0ab6a216569236433268b...
1,/en/block/history/10708/0b76b8f809ad5d8bcf3579...
2,/en/block/history/10709/c5f53ced72a23f333cc186...
3,/en/block/history/10710/ead1d1704b1600c795619e...
4,/en/block/history/10711/f558d3bc819c8ee5dc627d...
...,...
105,/en/block/history/11819/96c0eb3fa403ebf222f28b...
106,/en/block/history/11820/17fa8e795e69e9f326ea26...
107,/en/block/history/11821/a3fe7d350ef546e3c27ae8...
108,/en/block/history/11822/ca646e1b4b99d7ee973c45...


In [70]:
dfstagesrestmp = pd.DataFrame()
dfrankrestmp = pd.DataFrame()



for index, row in dflink_reversed.iterrows():
    url = PREFIX + row["TDFHistorylink"]
    try:
        if index >= 12:  # limit from 1919 (data need to be cleaned a little bit more before that)
            dfstagesres, dfrankres = getstagesNrank(url)
            dfstagesrestmp = pd.concat((dfstagesrestmp, dfstagesres), axis=0)
            dfrankrestmp = pd.concat((dfrankrestmp, dfrankres), axis=0)
    except Exception as e:
        print(f"An error occurred: {e}")
        raise

http://www.letour.fr/en/block/history/10719/f15cf81b7599ea113d2b1d614f73bf83 ==> HTTP STATUS = 200


http://www.letour.fr/en/block/history/10720/53ac40095834ae1aae5d197aa730799a ==> HTTP STATUS = 200


http://www.letour.fr/en/block/history/10721/c663ba39ac60d58c55a1a787a23025fc ==> HTTP STATUS = 200


http://www.letour.fr/en/block/history/10722/cc5d0ba48a5d9bd9ee5e222160452b6f ==> HTTP STATUS = 200


http://www.letour.fr/en/block/history/10723/9212137527c3f19c0cfccd1d07d80649 ==> HTTP STATUS = 200


...

http://www.letour.fr/en/block/history/11819/96c0eb3fa403ebf222f28b6e45115c56 ==> HTTP STATUS = 200


http://www.letour.fr/en/block/history/11820/17fa8e795e69e9f326ea26cf9912e571 ==> HTTP STATUS = 200


http://www.letour.fr/en/block/history/11821/a3fe7d350ef546e3c27ae83a84fa3a0b ==> HTTP STATUS = 200


http://www.letour.fr/en/block/history/11822/ca646e1b4b99d7ee973c457c87392395 ==> HTTP STATUS = 200


http://www.letour.fr/en/block/history/11823/4e4949d130f21278f7d8bce32a071d67 ==> HTTP STATUS = 200


In [71]:
dfstageoutput = dfstagesrestmp
dfrankoutput = dfrankrestmp

In [72]:
dfrankoutput

,Rank,Rider,Team,Times,Gap,B,P,Year,Distance (km),Number of stages,TotalSeconds,GapSeconds
0,1,FIRMIN LAMBOT,TDF 1919 ***,231h 07' 15'',-,NaN,NaN,1919,5560,15,832035,0
1,2,JEAN ALAVOINE,TDF 1919 ***,232h 50' 09'',+ 01h 42' 54'',NaN,NaN,1919,5560,15,838209,6174
2,3,EUGÈNE CHRISTOPHE,TDF 1919 ***,233h 33' 46'',+ 02h 26' 31'',NaN,NaN,1919,5560,15,840826,8791
3,4,LÉON SCIEUR,TDF 1919 ***,233h 59' 30'',+ 02h 52' 15'',NaN,NaN,1919,5560,15,842370,10335
4,5,HONORÉ BARTHELEMY,TDF 1919 ***,235h 21' 37'',+ 04h 14' 22'',NaN,NaN,1919,5560,15,847297,15262
...,...,...,...,...,...,...,...,...,...,...,...,...
145,146,ALEXANDER EDMONDSON,TEAM DSM - FIRMENICH,87h 50' 21'',+ 05h 44' 39'',NaN,NaN,2023,3404,21,316221,20679
146,147,FREDERIK FRISON,LOTTO DSTNY,88h 01' 02'',+ 05h 55' 20'',NaN,NaN,2023,3404,21,316862,21320
147,148,YEVGENIY FEDOROV,ASTANA QAZAQSTAN TEAM,88h 02' 19'',+ 05h 56' 37'',NaN,NaN,2023,3404,21,316939,21397
148,149,CEES BOL,ASTANA QAZAQSTAN TEAM,88h 03' 26'',+ 05h 57' 44'',NaN,NaN,2023,3404,21,317006,21464


## 3) Clean up the data 


In [73]:
# Fix result types
dfrankoutput["ResultType"] = "time"
dfrankoutput.loc[dfrankoutput["Year"].isin([1905, 1906, 1908]), "ResultType"] = "null"
dfrankoutput.loc[
    dfrankoutput["Year"].isin([1907, 1909, 1910, 1911, 1912]), "ResultType"
] = "points"

In [74]:
# Fix this weird bug for e.g. year 2006 and 1997
for year in np.unique(dfrankoutput["Year"]):
    tmp = dfrankoutput[dfrankoutput["Year"] == year].reset_index()
    if tmp.loc[0]["TotalSeconds"] > tmp.loc[2]["TotalSeconds"]:
        print(year)
# Okay seems to be only for 2006 and 1997

1997
2006


In [75]:
tmp = dfrankoutput[dfrankoutput["Year"] == 2006].reset_index()
ts = np.array(tmp["TotalSeconds"])
gs = np.array(tmp["GapSeconds"])
ts[1:] = ts[0] + gs[1:]

dfrankoutput.loc[dfrankoutput["Year"] == 2006, "TotalSeconds"] = ts

tmp = dfrankoutput[dfrankoutput["Year"] == 1997].reset_index()
ts = np.array(tmp["TotalSeconds"])
gs = np.array(tmp["GapSeconds"])
ts[1:] = ts[0] + gs[1:]

dfrankoutput.loc[dfrankoutput["Year"] == 1997, "TotalSeconds"] = ts

## 4) Write Data to CSV

In [76]:
dfrankoutput.to_csv("../data/TDF_Riders_History.csv")

In [77]:
dfstageoutput.to_csv("../data/TDF_Stages_History.csv")